# Speech Identity Inference

Let's check if the pretrained model can really identify speakers.

In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from IPython.display import Audio
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_addons as tfa

from train_speech_id_model import BaseSpeechEmbeddingModel
from create_audio_tfrecords import AudioTarReader, PersonIdAudio

sr = 48000

In [ ]:
m = BaseSpeechEmbeddingModel()
m.summary()

In [ ]:
m.load_weights('temp/cp-0060.ckpt')

In [ ]:
# changing the corpus to other languages allows evaluating how the model transfers between languages
dev_dataset = tfrecords_audio_dataset = tf.data.TFRecordDataset(
    'data/cv-corpus-7.0-2021-07-21-pt.tar.gz_dev.tfrecords.gzip', compression_type='GZIP',
    num_parallel_reads=4
).map(PersonIdAudio.deserialize_from_tfrecords)

In [ ]:
samples = [x for x in dev_dataset.take(1000)]
# decode audio
samples = [(tfio.audio.decode_mp3(x[0])[:, 0], x[1]) for x in samples]

In [ ]:
# is the audio decoded correctly?
Audio(samples[10][0], rate=sr)

In [ ]:
# compute the embeddings
embeddings = []
for audio_data, person_id in tqdm(samples):
    cur_emb = m.predict(
        tf.expand_dims(audio_data, axis=0)
    )
    embeddings.append(cur_emb)

## Check embedding quality

Ideally, embeddings from the same person should look the same.

In [ ]:
n_speakers = len(set([x[1].numpy() for x in samples]))
print(f'Loaded {n_speakers} different speakers')

In [ ]:
pairwise_diff = {'same': [], 'different': []}
for p in tqdm(range(len(samples))):
    for q in range(p + 1, len(samples)):
        id_1 = samples[p][1]
        id_2 = samples[q][1]
        dist = np.linalg.norm(embeddings[p] - embeddings[q])
        if id_1 == id_2:
            pairwise_diff['same'].append(dist)
        else:
            pairwise_diff['different'].append(dist)

In [ ]:
plt.boxplot([pairwise_diff[x] for x in pairwise_diff])
plt.xticks([k + 1 for k in range(len(pairwise_diff))], [x for x in pairwise_diff])
plt.ylabel('Embedding distance')
plt.title('Boxplot of speaker identifiability')

In [ ]:
# what do we care about?
# given that 2 samples are different, we don't want to predict `same`
# secondarily, given that 2 samples are the same, we want to predict `same`

# threshold
t = 0.2 * np.median(pairwise_diff['different']) + np.median(pairwise_diff['same']) * 0.8

specificity = np.sum(np.array(pairwise_diff['different'] > t)) / len(pairwise_diff['different'])
sensitivity = np.sum(np.array(pairwise_diff['same'] < t)) / len(pairwise_diff['same'])

print('Sensitivity, specificity = ', sensitivity, specificity)

same_lbl = [0] * len(pairwise_diff['same'])
diff_lbl = [1] * len(pairwise_diff['different'])
scores = pairwise_diff['same'] + pairwise_diff['different']

scores = np.array(scores) * 0.5
t = t * 0.5

labels = same_lbl + diff_lbl
len(scores), len(labels)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(labels, scores, pos_label=1)

In [ ]:
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1])

In [ ]:
plt.title('Point of operation')
plt.plot(thresholds, 1 - fpr, label='Specificity')
plt.plot(thresholds, tpr, label='Sensitivity')
plt.plot([t, t], [0, 1], label='Threshold')
plt.xlabel('Threshold level')
plt.legend()

## Select best model on validation

Strategy: compute loss but don't sort validation set, so there are multiple voice repeats in a batch. Also makes the evaluation consistent. Batch size should be as big as possible.

In [ ]:
triplet_loss = tfa.losses.TripletSemiHardLoss()

In [ ]:
# compute all predictions
def mp3_decode_fn(audio_bytes, audio_class):
    audio_data = tfio.audio.decode_mp3(audio_bytes)[:, 0]
    return audio_data, audio_class

all_preds = []
all_labels = []
for x in tqdm(dev_dataset.take(1300).map(mp3_decode_fn)):
    s = x[0]
    all_preds.append(m.predict(
        tf.expand_dims(x[0], axis=0)
    )[0])
    all_labels.append(x[1].numpy())

In [ ]:
len(all_preds)

In [ ]:
batch_size = 128
n_batches = len(all_preds) // batch_size
vec_size = len(all_preds[0])

np_preds = np.reshape(all_preds[0:batch_size * n_batches], (n_batches, batch_size, vec_size))
np_labls = np.reshape(all_labels[0:batch_size * n_batches], (n_batches, batch_size))

In [ ]:
total_loss = 0
for lbl, pred in zip(np_labls, np_preds):
    total_loss += triplet_loss(lbl, pred).numpy()
total_loss = total_loss / len(lbl)
print(f'Total loss: {total_loss}')